In [116]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [117]:
column_names = ['second', 'function_id', 'inference_time']
baseline = pd.read_csv('../../results/baseline/evaluation_results.txt', header=None, names=column_names)
column_names = ['second', 'function_id', 'inference_time', 'preloaded_function_id_model']
experiment_2 = pd.read_csv('../../results/experiment2/evaluation_results.txt', header=None, names=column_names)

In [118]:
baseline

,second,function_id,inference_time
0,950400,1,0.293437
1,950400,56,0.176041
2,950400,69,0.176354
3,950400,71,0.179602
4,950401,1,0.176239
...,...,...,...
23731,1209302,50,0.118754
23732,1209363,55,0.118659
23733,1209424,31,0.118680
23734,1209432,37,0.123342


In [119]:
experiment_2

,second,function_id,inference_time,preloaded_function_id_model
0,950400,1,0.266609,1
1,950400,56,0.156542,1
2,950400,69,0.173575,1
3,950400,71,0.174598,1
4,950401,1,0.024378,1
...,...,...,...,...
23731,1209302,50,0.124165,56
23732,1209363,55,0.026125,55
23733,1209424,31,0.123983,21
23734,1209432,37,0.124386,21


In [120]:
baseline.function_id.unique()

array([ 1, 56, 69, 71, 50, 31, 55, 37, 21,  7])

In [121]:
filtered_1 = baseline[baseline['function_id'] == 1]
filtered_56 = baseline[baseline['function_id'] == 56]
filtered_69 = baseline[baseline['function_id'] == 69]
filtered_71 = baseline[baseline['function_id'] == 71]
filtered_50 = baseline[baseline['function_id'] == 50]
filtered_31 = baseline[baseline['function_id'] == 31]
filtered_55 = baseline[baseline['function_id'] == 55]
filtered_37 = baseline[baseline['function_id'] == 37]
filtered_21 = baseline[baseline['function_id'] == 21]
filtered_7 = baseline[baseline['function_id'] == 7]

In [122]:
ex2_filtered_1 = experiment_2[experiment_2['function_id'] == 1]
ex2_filtered_56 = experiment_2[experiment_2['function_id'] == 56]
ex2_filtered_69 = experiment_2[experiment_2['function_id'] == 69]
ex2_filtered_71 = experiment_2[experiment_2['function_id'] == 71]
ex2_filtered_50 = experiment_2[experiment_2['function_id'] == 50]
ex2_filtered_31 = experiment_2[experiment_2['function_id'] == 31]
ex2_filtered_55 = experiment_2[experiment_2['function_id'] == 55]
ex2_filtered_37 = experiment_2[experiment_2['function_id'] == 37]
ex2_filtered_21 = experiment_2[experiment_2['function_id'] == 21]
ex2_filtered_7 = experiment_2[experiment_2['function_id'] == 7]

Below is missed improvement code to get how many times a correct model was chosen for preloading but this iprovement was not taken into accouint because of specifics of the experiment that all requests for one second were performend sequentially.

In [123]:
counts = experiment_2['second'].value_counts()
df = experiment_2[experiment_2['second'].isin(counts[counts > 1].index)]

missed_improvement = []
index = 0
for _, group in df.groupby('second'):
    for _, row in group.iterrows():
        if index == 0:
            missed_improvement.append(0)
            index += 1
            continue
        if row['preloaded_function_id_model'] == row['function_id']:
            missed_improvement.append(1)
        else:
            missed_improvement.append(0)
        index += 1
    index = 0

df['missed_improvement'] = missed_improvement

/var/folders/bv/mvwnb3fd1rdbr345qwm8mqb40000gn/T/ipykernel_70979/1706891020.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [124]:
df

,second,function_id,inference_time,preloaded_function_id_model,missed_improvement
0,950400,1,0.266609,1,0
1,950400,56,0.156542,1,0
2,950400,69,0.173575,1,0
3,950400,71,0.174598,1,0
4,950401,1,0.024378,1,0
...,...,...,...,...,...
23656,1206544,31,0.123505,1,0
23657,1206552,1,0.025568,1,0
23658,1206552,37,0.123736,1,0
23671,1207084,31,0.123914,55,0


In [125]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_1['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_1['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 1", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [126]:
print(f"Average inference time for baseline: {filtered_1['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_1['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 1]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

Average inference time for baseline: 0.11951481875456393
Average inference time for preloading: 0.048166979298341434
Amount of missed improvements: 0


In [127]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_7['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_7['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 7", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [128]:
print(f"Average inference time for baseline: {filtered_7['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_7['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 7]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

selected_indices = ex2_filtered_7[(ex2_filtered_7['inference_time'] > 0.1)].sample(n=736, random_state=42).index
ex2_filtered_7.loc[selected_indices, 'inference_time'] = 0.024
print(f"Average inference time for preloading (after including influence of 'missed improvements'): {ex2_filtered_7['inference_time'].mean()}")

Average inference time for baseline: 0.05309907330622231
Average inference time for preloading: 0.061761821870586875
Amount of missed improvements: 736
Average inference time for preloading (after including influence of 'missed improvements'): 0.05434505780318417


In [129]:
ex2_filtered_7

,second,function_id,inference_time,preloaded_function_id_model
1641,983468,7,0.024000,1
1643,983469,7,0.118867,1
1645,983470,7,0.118859,1
1647,983471,7,0.119261,1
1649,983472,7,0.121905,1
...,...,...,...,...
23143,1190905,7,0.025059,7
23144,1190908,7,0.025169,7
23145,1190909,7,0.025779,7
23146,1190910,7,0.025200,7


In [130]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_21['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_21['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 21", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [131]:
print(f"Average inference time for baseline: {filtered_21['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_21['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 21]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

Average inference time for baseline: 0.10764035575753732
Average inference time for preloading: 0.11650416927076958
Amount of missed improvements: 0


In [132]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_31['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_31['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 31", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [133]:
print(f"Average inference time for baseline: {filtered_31['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_31['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 31]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

Average inference time for baseline: 0.11983652121481712
Average inference time for preloading: 0.12326303665912429
Amount of missed improvements: 0


In [134]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_37['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_37['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 37", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [135]:
print(f"Average inference time for baseline: {filtered_37['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_37['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 37]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

selected_indices = ex2_filtered_37[(ex2_filtered_37['inference_time'] > 0.1)].sample(n=4, random_state=42).index
ex2_filtered_37.loc[selected_indices, 'inference_time'] = 0.024
print(f"Average inference time for preloading (after including influence of 'missed improvements'): {ex2_filtered_37['inference_time'].mean()}")

Average inference time for baseline: 0.1195318910223625
Average inference time for preloading: 0.11878175467995722
Amount of missed improvements: 4
Average inference time for preloading (after including influence of 'missed improvements'): 0.11850733855820755


In [136]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_50['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_50['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 50", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [137]:
print(f"Average inference time for baseline: {filtered_50['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_50['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 50]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

selected_indices = ex2_filtered_50[(ex2_filtered_50['inference_time'] > 0.1)].sample(n=2, random_state=42).index
ex2_filtered_50.loc[selected_indices, 'inference_time'] = 0.024
print(f"Average inference time for preloading (after including influence of 'missed improvements'): {ex2_filtered_50['inference_time'].mean()}")

Average inference time for baseline: 0.11977461394336485
Average inference time for preloading: 0.12260216060611934
Amount of missed improvements: 2
Average inference time for preloading (after including influence of 'missed improvements'): 0.12237224219905002


In [138]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_55['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_55['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 55", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [139]:
print(f"Average inference time for baseline: {filtered_55['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_55['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 55]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

selected_indices = ex2_filtered_55[(ex2_filtered_55['inference_time'] > 0.1)].sample(n=85, random_state=42).index
ex2_filtered_55.loc[selected_indices, 'inference_time'] = 0.024
print(f"Average inference time for preloading (after including influence of 'missed improvements'): {ex2_filtered_55['inference_time'].mean()}")

Average inference time for baseline: 0.11836916282738535
Average inference time for preloading: 0.09248048450482323
Amount of missed improvements: 85
Average inference time for preloading (after including influence of 'missed improvements'): 0.08860393887369344


In [140]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_56['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_56['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 56", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [141]:
print(f"Average inference time for baseline: {filtered_56['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_56['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 56]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

selected_indices = ex2_filtered_56[(ex2_filtered_56['inference_time'] > 0.1)].sample(n=10, random_state=42).index
ex2_filtered_56.loc[selected_indices, 'inference_time'] = 0.024
print(f"Average inference time for preloading (after including influence of 'missed improvements'): {ex2_filtered_56['inference_time'].mean()}")

Average inference time for baseline: 0.11989338668408214
Average inference time for preloading: 0.10863344233345099
Amount of missed improvements: 10
Average inference time for preloading (after including influence of 'missed improvements'): 0.1074338991222558


In [142]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_69['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_69['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 69", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [143]:
print(f"Average inference time for baseline: {filtered_69['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_69['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 69]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

selected_indices = ex2_filtered_69[(ex2_filtered_69['inference_time'] > 0.1)].sample(n=35, random_state=42).index
ex2_filtered_69.loc[selected_indices, 'inference_time'] = 0.024
print(f"Average inference time for preloading (after including influence of 'missed improvements'): {ex2_filtered_69['inference_time'].mean()}")

Average inference time for baseline: 0.12039022870583106
Average inference time for preloading: 0.08253481895616735
Amount of missed improvements: 35
Average inference time for preloading (after including influence of 'missed improvements'): 0.07402043493667447


In [144]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Baseline", "Preloading"))

histogram_baseline = go.Histogram(x=filtered_71['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_baseline, row=1, col=1)

histogram_preloading = go.Histogram(x=ex2_filtered_71['inference_time'], nbinsx=100, histnorm='')
fig.add_trace(histogram_preloading, row=1, col=2)

fig.update_layout(title_text="Histogram of Inference Time for Function ID 71", showlegend=False)

fig.update_xaxes(range=[0, 0.2], row=1, col=1)
fig.update_xaxes(range=[0, 0.2], row=1, col=2)

fig.show()

In [145]:
print(f"Average inference time for baseline: {filtered_71['inference_time'].mean()}")
print(f"Average inference time for preloading: {ex2_filtered_71['inference_time'].mean()}")

tmp = df.loc[df["function_id"] == 71]
print(f"Amount of missed improvements: {tmp.missed_improvement.sum()}")

selected_indices = ex2_filtered_71[(ex2_filtered_71['inference_time'] > 0.1)].sample(n=25, random_state=42).index
ex2_filtered_71.loc[selected_indices, 'inference_time'] = 0.024
print(f"Average inference time for preloading (after including influence of 'missed improvements'): {ex2_filtered_71['inference_time'].mean()}")

Average inference time for baseline: 0.1097576558589935
Average inference time for preloading: 0.09443601767222083
Amount of missed improvements: 25
Average inference time for preloading (after including influence of 'missed improvements'): 0.08773482335938344
